The kernel is based on the Candidate Generation one with a few modifications and parts that have been rewritten to work a little bit better.

**Reading a CT Scan**
-----------------
The input folder has three things, one is the sample_images folders which has the sample CT Scans. The `stage1_labels.csv` contains the cancer ground truth for the stage 1 training set images and `stage1_sample_submission.csv` shows the submission format for stage 1. 

In [2]:
import numpy as np # linear algebra
import pandas as pd # reading and processing of tables
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage.util.montage import montage2d
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
CT_OFFSET = 1024
ZERO_VALUE = -2000

In [3]:
# we want to crop out the regions we dont need first
def _dsum(carr,  # type: np.ndarray
          cax  # type: int
          ):
    # type: (np.ndarray, int) -> np.ndarray
    return np.sum(carr, tuple(n for n in range(carr.ndim) if n is not cax))

def get_bbox(in_vol,
             min_val=0):
    # type: (np.ndarray, float) -> List[Tuple[int,int]]
    ax_slice = []
    for i in range(in_vol.ndim):
        c_dim_sum = _dsum(in_vol > min_val, i)
        wh_idx = np.where(c_dim_sum)[0]
        c_sl = sorted(wh_idx)
        if len(wh_idx) == 0:
            ax_slice += [(0, 0)]
        else:
            ax_slice += [(c_sl[0], c_sl[-1] + 1)]
    return ax_slice

def apply_bbox(in_vol,  # type: np.ndarray
               bbox_list,  # type: List[Tuple[int,int]]
               pad_values = False,
               padding_mode = 'edge'
               ):
    return in_vol.__getitem__([slice(a, b, 1) for (a, b) in bbox_list])

def autocrop(in_vol,  # type: np.ndarray
             min_val  # type: double
             ):
    return apply_bbox(in_vol, get_bbox(in_vol,
                                       min_val=min_val))

Each 3D CT Scan consists of many slices, whose number depends on the resolution of the scanner and each slice has a Instance Number associated with it which tells the index of the slice from the top. All the dicom files for a CT Scan are inside one folder having the CT Scan's name. Now we will read all the dicom slices for a scan and then stack them with respect to their Instance Number to get the 3D Lung CT Scanned Image.

In [4]:
image_tags_df = pd.read_csv('../input/stage1_labels.csv')
image_tags_df.sample(3)

Each CT Scan consists of multiple 2D slices which are provided in a DICOM format. At first, I will read the random dicom file of a CT Scan. After reading the image file, we will update the intensity values of -2000 with 0 because they are the pixels that fall outside of the scanner bounds.

In [5]:
# Read an image and the image path
def read_dicom_array(in_path):
    # type: (str) -> Tuple[int, np.ndarray]
    lung_dicom = dicom.read_file(in_path)
    slice_array = lung_dicom.pixel_array
    slice_array[slice_array == -2000] = 0
    return int(lung_dicom.InstanceNumber), slice_array.astype(np.int16) - CT_OFFSET
_, slice_arr = read_dicom_array('../input/sample_images/00cba091fa4ad62cc3200a657aeb957e/38c4ff5d36b5a6b6dc025435d62a143d.dcm')

c_img_ax = plt.imshow(slice_arr, cmap=plt.cm.bone)
plt.colorbar(c_img_ax)

In [6]:
def read_ct_scan(folder_name):
        # Read the slices from the dicom file
        slices = [read_dicom_array(folder_name + filename) for filename in os.listdir(folder_name)]
        
        # Sort the dicom slices in their respective order
        s_slices = sorted(slices, key = lambda x: x[0])
        
        # Get the pixel values for all the slices
        slices = np.stack([data for pos, data in s_slices])
        return slices

In [7]:
ct_scan = read_ct_scan('../input/sample_images/00cba091fa4ad62cc3200a657aeb957e/') 
print('Scan Dimensions',ct_scan.shape)

To visualise the slices, we will have to plot them. `matplotlib` is used for plotting the slices. The `plot_ct_scan` function takes a 3D CT Scanned Image array  as input and plots equally spaced slices. The CT Scans are grayscale images i.e. the value of each pixel is a single sample, which means it carries only intensity information.

In [8]:
def plot_ct_scan(scan):
    """
    show all the slices the ct as a montage
    """
    f, ax1 = plt.subplots(1,1, figsize=(12, 12))
    ax1.imshow(montage2d(scan), cmap=plt.cm.bone) 
    ax1.axis('off')

In [9]:
plot_ct_scan(ct_scan)

## Segmentation of Lungs
After reading the CT Scan, the first step in preprocessing is the segmentation of lung structures because it is obvious that the regions of interests lies inside the lungs. It is visible that the lungs are the darker regions in the CT Scans. The bright region inside the lungs are the blood vessels or air. A threshold of (-400 HU) is used at all places because it was found in experiments that it works just fine. We segment lung structures from each slice of the CT Scan image and try not to loose the possible region of interests attached to the lung wall. There are some nodules which may be attached to the lung wall.

I will first explain a common method using simple Image Processing and Morphological operations to segment the lungs and then will give references and summaries to good links of papers. 

In [11]:
def get_segmented_lungs(in_im, plot=False):
    im = in_im.copy() # don't change the input
    '''
    This funtion segments the lungs from the given 2D slice.
    '''
    if plot == True:
        f, plots = plt.subplots(3, 3, figsize=(10, 10))
        plots = plots.flatten()
    '''
    Step 1: Convert into a binary image. 
    '''
    binary = im < -400
    if plot == True:
        plots[0].axis('off')
        plots[0].imshow(binary, cmap=plt.cm.bone) 
        plots[0].set_title('First Threshold')
    '''
    Step 2: Remove the blobs connected to the border of the image.
    '''
    cleared = clear_border(binary)
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(cleared, cmap=plt.cm.bone) 
        plots[1].set_title('Remove Border')
    '''
    Step 3: Label the image.
    '''
    label_image = label(cleared)
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(label_image, cmap=plt.cm.gist_earth)
        plots[2].set_title('Label Components')
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary, cmap=plt.cm.bone) 
        plots[3].set_title('Keep Biggest 2')
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(2)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(binary, cmap=plt.cm.bone)
        plots[4].set_title('Erosion')
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
        plots[5].set_title('Close Image')
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''
    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
        plots[6].set_title('Fill holes')
    '''
    Step 8: Superimpose the binary mask on the input image.
    '''
    get_high_vals = (binary == 0)
    im[get_high_vals] = ZERO_VALUE # minimum value
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(im, cmap=plt.cm.bone) 
        plots[7].set_title('Binary Masked Input')
        
    return im

The `get_segmented_lungs` function segments a 2D slice of the CT Scan. I have outputted the slice after all steps for better visualisation and understanding of the code and applied operations.

In [12]:
get_segmented_lungs(ct_scan[71], True)

Now, I will segment the whole CT Scan slice by slice and show some slices of the CT Scan.

In [13]:
def segment_lung_from_ct_scan(ct_scan):
    return np.stack([get_segmented_lungs(c_slice) for c_slice in ct_scan], 0)

In [14]:
segmented_ct_scan = segment_lung_from_ct_scan(ct_scan)
plot_ct_scan(segmented_ct_scan)

# Super Pixel Generation
Here we generate superpixels for the regions the lungs

In [56]:
# Renormalize Image
mean_val = ct_proj[n_ct_img>ZERO_VALUE].mean()
sd_val = ct_proj[n_ct_img>ZERO_VALUE].std()
print(mean_val, sd_val)
n_ct_img = (ct_proj-mean_val)/(3*sd_val)
n_ct_img = (n_ct_img+0.1).clip(0,1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (8, 4))
ax1.imshow(n_ct_img, vmin = 0, vmax = 1, cmap = 'bone')
ax1.set_title('Rescaled Image')
_ = ax2.hist(n_ct_img.flatten(), np.linspace(0,1,30))
ax2.set_title('Histogram')

In [57]:
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
ct_proj = segmented_ct_scan[74]
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15, 10))

ct_segs = slic(n_ct_img, 
               n_segments = 200, 
               compactness = 0.02)

ax1.imshow(ct_proj, cmap = 'bone')
ax1.set_title('CT Image')

ct_segs[ct_proj==ZERO_VALUE] = 0
ax2.imshow(ct_segs, cmap = plt.cm.RdBu)
ax2.set_title('Segmented Image')

ax3.imshow(mark_boundaries(n_ct_img, ct_segs))
ax3.set_title('Visible Boundaries')

## Calculate the entire stack Superpixels
Here we calculate the superpixels for the entire stack, we then reorder it so it is easier to see in the colormaps

In [70]:
norm_lung_stack = (segmented_ct_scan-mean_val)/(3*sd_val)
norm_lung_stack = (norm_lung_stack+0.1).clip(0,1)

nl_segs = slic(norm_lung_stack, 
               n_segments = 200, 
               compactness = 0.1,
              multichannel = False) # otherwise it assumes a color image

nl_segs[segmented_ct_scan==ZERO_VALUE] = 0 # remove unneeded segments
# reorder segments
nl_segs_new = np.zeros_like(nl_segs)
for i_new, i_old in enumerate(sorted(np.unique(nl_segs))):
    if i_new>0:
        nl_segs_new[nl_segs==i_old] = i_new

In [66]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (15, 25))

ax1.imshow(montage2d(nl_segs_new), cmap = plt.cm.rainbow)
ax1.set_title('Segmented Image')

ax2.imshow(mark_boundaries(montage2d(norm_lung_stack), 
                           montage2d(nl_segs_new).astype(nl_segs_new.dtype)))
ax2.set_title('Visible Boundaries')

Skip slices to get a better overview

In [69]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (15, 25))

ax1.imshow(montage2d(nl_segs_new[::4]), cmap = plt.cm.rainbow, interpolation = 'none')
ax1.set_title('Segmented Image')

ax2.imshow(mark_boundaries(montage2d(norm_lung_stack[::4]), 
                           montage2d(nl_segs_new[::4]).astype(nl_segs_new.dtype)),
          interpolation = 'none')
ax2.set_title('Visible Boundaries')

Nodule Candidate/Region of Interest Generation
---------------------------
After segmenting the lung structures from the CT Scanned images, our task is to find the candidate regions with nodules since the search space is very large. Also, whole image cant be classified directly using 3D CNNs due to limit on computation, we need to find possible regions of cancer and then classify them. It was found in experiments that all the region of interests have intensity >  -400 HU. So, we used this threshold to filter the darker regions. This reduces the number of candidates by a large number and preserves all the important regions with high recall. We then classify all the candidate points to reduce the False Positives.

In [ ]:
segmented_ct_scan[segmented_ct_scan < -400] = ZERO_VALUE
plot_ct_scan(autocrop(segmented_ct_scan, ZERO_VALUE))

After filtering, there are still lot of noise because of blood vessels. Thus we further remove the two largest connected component.

In [ ]:
selem = ball(2)
binary = binary_closing(segmented_ct_scan, selem)

label_scan = label(binary)

areas = [r.area for r in regionprops(label_scan)]
areas.sort()

for r in regionprops(label_scan):
    max_x, max_y, max_z = 0, 0, 0
    min_x, min_y, min_z = label_scan.shape # use the shape instead of hard-coded values
    
    for c in r.coords:
        max_z = max(c[0], max_z)
        max_y = max(c[1], max_y)
        max_x = max(c[2], max_x)
        
        min_z = min(c[0], min_z)
        min_y = min(c[1], min_y)
        min_x = min(c[2], min_x)
    if (min_z == max_z or min_y == max_y or min_x == max_x or r.area > areas[-3]):
        for c in r.coords:
            segmented_ct_scan[c[0], c[1], c[2]] = 0
    else:
        index = (max((max_x - min_x), (max_y - min_y), (max_z - min_z))) / (min((max_x - min_x), (max_y - min_y) , (max_z - min_z)))

The `plot_3d` function plots the 3D numpy array of CT Scans. 

In [ ]:
def plot_3d(image, threshold):
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    p = p[:,:,::-1]
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [ ]:
plot_3d(autocrop(segmented_ct_scan, -400), -400)

### Show the individual components one at a time

In [ ]:
cbase_image = autocrop(segmented_ct_scan, -400)
cur_labels = label(base_image>-400)
max_comp = np.max(cur_labels)
comp_by_size = reversed(np.argsort([0]+[np.sum(cur_labels==i) for i in range(1, max_comp+1)]))
print('Components Found:', max_comp)
fig_count = 0
for i in comp_by_size:
    c_bbox = get_bbox(cur_labels==i)  
    print('Component:', i, c_bbox)
    cur_obj = apply_bbox(base_image, c_bbox)
    if (cur_obj.shape[0]>4) & (cur_obj.shape[1]>4)  & (cur_obj.shape[2]>4):
        print(i, cur_obj.min(), cur_obj.max())
        plot_3d(cur_obj, np.median([ cur_obj.min(), -400,  cur_obj.max()]))
        fig_count +=1
    if fig_count>3: # maximum figure count
        break


**Please upvote or leave a comment, if you liked the tutorial.**